In [ ]:
!pip install feedparser
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade tensorflow
!pip install git+https://github.com/urllib3/urllib3.git
!pip install transformers

In [ ]:
import pandas as pd
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import time
from transformers import pipeline
from tqdm import tqdm
import re

In [ ]:
dfclients = pd.read_excel(r'Destinos.xlsx')
d=dfclients['Destinos']
d.values.tolist()
d = d.replace("++","+")
series= pd.Series(d)
series = series.str.strip('+')

In [ ]:
destinos = series.values.tolist()
destinos

['Buenos+Aires', 'Mar+del+Plata', 'Cordoba', 'Mendoza', 'Bariloche']

In [ ]:
#Scrap news
destinolist = []

class ParseFeed():
 
    def __init__(self, url):
        self.feed_url = url

    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text
 
    def parse(self):
        '''
        Parse the URL, and print all the details of the news 
        '''
        feeds = feedparser.parse(self.feed_url).entries
        for f in feeds:
              destinolist.append({
                'Description': self.clean(f.get("description", "")),
                'Published Date': f.get("published", ""),
                'Title': f.get("title", ""),
                'Url': f.get("link", "")
              })
              break
        print(destinolist)


def empty():
    destinolist[:] = []

In [ ]:
#Lists with keywords
words= ["clima","Temperatura+promedio+anual","Lugares+para+visitar","Arquitectura","Platos","Comida+tipica","Bebida+tipica","Atracciones","Oferta+hotelera","Cantidad+habitantes","Que+hacer?","en+que+epoca+ir","cuantos+dias+visitar","como+recorrer","Actividades+para+hacer","Oferta+gastronomica","Transporte","Costo+para+vacacionar","pronostico+clima" ]


In [ ]:
#creation of df in blank with columns to fill
scrap_df= pd.DataFrame(columns = ["Description", "Published Date", "Title", "Url","search_var"])
#loop first for customers, after for words and if the word is "neue" it loops with the specific words. If not is only customer-word.
for destino in destinos:
  for word in words:
    word = str(word.encode('utf-8'))[2:-1]
    url = "https://news.google.com.ar/rss/search?q="+destino+"+"+word+"&hl=es-ES&sort=date&gl=AR&num=100&output=rss"
    print(url)
    feed = ParseFeed(url)
    feed.parse()
    df=pd.DataFrame(destinolist,columns=["Description","Published Date", "Title","Url"])
    df["destino"]=destino
    df["search_var"]=destino +'+'+ word
    scrap_df=scrap_df.append(df,ignore_index=True)
    empty()
    time.sleep(0.01)

https://news.google.com.ar/rss/search?q=Buenos+Aires+clima&hl=es-ES&sort=date&gl=AR&num=100&output=rss
[{'Description': 'Clima en Buenos Aires: el pronóstico del tiempo para este domingo 20 de febrero  Página 12Clima en Buenos Aires: el pronóstico del tiempo para este lunes 21 de febrero  750.amClima en la Ciudad de Buenos Aires: lunes 21 de febrero  WeekendClima en Buenos Aires: pronóstico del domingo 13 de febrero de 2022  ámbito.comDomingo con lluvias y una máxima de 26 grados en ciudad de Buenos Aires y el conurbano  TélamVer cobertura completa en Google Noticias', 'Published Date': 'Sun, 20 Feb 2022 13:23:48 GMT', 'Title': 'Clima en Buenos Aires: el pronóstico del tiempo para este domingo 20 de febrero - Página 12', 'Url': 'https://www.pagina12.com.ar/402762-clima-en-buenos-aires-el-pronostico-del-tiempo-para-este-dom'}]
https://news.google.com.ar/rss/search?q=Buenos+Aires+Temperatura+promedio+anual&hl=es-ES&sort=date&gl=AR&num=100&output=rss
[{'Description': 'El impacto del clima

In [ ]:
scrap_df

,Description,Published Date,Title,Url,search_var,destino
0,Clima en Buenos Aires: el pronóstico del tiemp...,"Sun, 20 Feb 2022 13:23:48 GMT",Clima en Buenos Aires: el pronóstico del tiemp...,https://www.pagina12.com.ar/402762-clima-en-bu...,Buenos+Aires+clima,Buenos+Aires
1,El impacto del clima: el campo pasó de la sequ...,"Mon, 24 Jan 2022 08:00:00 GMT",El impacto del clima: el campo pasó de la sequ...,https://www.infobae.com/economia/campo/2022/01...,Buenos+Aires+Temperatura+promedio+anual,Buenos+Aires
2,Ocho escapadas de fin de semana en la provinci...,"Fri, 18 Feb 2022 12:58:33 GMT",Ocho escapadas de fin de semana en la provinci...,https://tn.com.ar/general/2022/02/18/ocho-esca...,Buenos+Aires+Lugares+para+visitar,Buenos+Aires
3,"¿Arquitectura sin arquitectos? Campanópolis, l...","Sun, 13 Feb 2022 17:00:00 GMT","¿Arquitectura sin arquitectos? Campanópolis, l...",https://www.plataformaarquitectura.cl/cl/97602...,Buenos+Aires+Arquitectura,Buenos+Aires
4,¿Cuáles son los mejores restaurantes de Buenos...,"Thu, 27 Jan 2022 08:00:00 GMT",¿Cuáles son los mejores restaurantes de Buenos...,https://www.lanacion.com.ar/revista-ohlala/cua...,Buenos+Aires+Platos,Buenos+Aires
...,...,...,...,...,...,...
90,Una guía de actividades para hacer en la Costa...,"Wed, 02 Feb 2022 08:00:00 GMT",Una guía de actividades para hacer en la Costa...,https://www.clarin.com/brandstudio/guia-activi...,Bariloche+Actividades+para+hacer,Bariloche
91,Pocos días faltan para la Expo Rural 2022 Bar...,"Mon, 14 Feb 2022 19:20:51 GMT",Pocos días faltan para la Expo Rural 2022 - Ba...,https://www.bariloche2000.com/noticias/leer/po...,Bariloche+Oferta+gastronomica,Bariloche
92,Convocan a una asamblea popular por el transpo...,"Fri, 04 Feb 2022 08:00:00 GMT",Convocan a una asamblea popular por el transpo...,https://www.rionegro.com.ar/politica/convocan-...,Bariloche+Transporte,Bariloche
93,Cuánto cuesta vacacionar en Bariloche este ver...,"Mon, 24 Jan 2022 08:00:00 GMT",Cuánto cuesta vacacionar en Bariloche este ver...,https://www.airedesantafe.com.ar/turismo/cuant...,Bariloche+Costo+para+vacacionar,Bariloche


In [ ]:
#duplicates
scrap_df= scrap_df.drop_duplicates(subset='Title', keep='first')
len(scrap_df)

77

In [ ]:
classifier = pipeline("zero-shot-classification")
listadonoticias=[]
candidate_labels = ["Buenos Aires", "Cordoba","Mar del Plata"," Mendoza","Bariloche","vacaciones","destinos","turismo","actividades","que hacer","eventos","vacacionales"]
candidate_results = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for i in tqdm(scrap_df['Title'].values):
    # To do multi-class classification, simply pass multi_class=True.
    # In this case, the scores will be independent, but each will fall between 0 and 1.
    res = classifier(i, candidate_labels)
    if res['scores'][0] > 0.20:
        listadonoticias.append([i,res['labels'][0],res['scores'][0]])
        

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

100%|██████████| 77/77 [12:08<00:00,  9.46s/it]


In [ ]:
print(listadonoticias)

[['Clima en Buenos Aires: el pronóstico del tiempo para este domingo 20 de febrero - Página 12', 'Buenos Aires', 0.7854580879211426], ['El impacto del clima: el campo pasó de la sequía a inundaciones, pero hay un mejor panorama para la soja y el maíz - infobae', 'eventos', 0.4520879089832306], ['Ocho escapadas de fin de semana en la provincia de Buenos Aires - TN - Todo Noticias', 'Buenos Aires', 0.3846818208694458], ['¿Arquitectura sin arquitectos? Campanópolis, la pequeña aldea medieval que nació del reciclado en Buenos Aires - Plataforma Arquitectura', 'Buenos Aires', 0.7346512675285339], ['¿Cuáles son los mejores restaurantes de Buenos Aires para disfrutar de platos con frutos del mar? - LA NACION', 'Buenos Aires', 0.7251686453819275], ['El sábado llega el Festival BA Capital Gastronómica - buenosaires.gob.ar', 'Buenos Aires', 0.6470471024513245], ['La Ciudad recibió el año 4.720 junto a la colectividad china - buenosaires.gob.ar', 'Buenos Aires', 0.6042594909667969], ['Feriado de 

In [ ]:
dfnews = pd.DataFrame(listadonoticias, columns=['Title', 'label', 'score'])

In [ ]:
scrap_df = pd.merge(scrap_df, dfnews, on='Title')

In [ ]:
scrap_df

,Description,Published Date,Title,Url,search_var,destino,label,score
0,Clima en Buenos Aires: el pronóstico del tiemp...,"Sun, 20 Feb 2022 13:23:48 GMT",Clima en Buenos Aires: el pronóstico del tiemp...,https://www.pagina12.com.ar/402762-clima-en-bu...,Buenos+Aires+clima,Buenos+Aires,Buenos Aires,0.785458
1,El impacto del clima: el campo pasó de la sequ...,"Mon, 24 Jan 2022 08:00:00 GMT",El impacto del clima: el campo pasó de la sequ...,https://www.infobae.com/economia/campo/2022/01...,Buenos+Aires+Temperatura+promedio+anual,Buenos+Aires,eventos,0.452088
2,Ocho escapadas de fin de semana en la provinci...,"Fri, 18 Feb 2022 12:58:33 GMT",Ocho escapadas de fin de semana en la provinci...,https://tn.com.ar/general/2022/02/18/ocho-esca...,Buenos+Aires+Lugares+para+visitar,Buenos+Aires,Buenos Aires,0.384682
3,"¿Arquitectura sin arquitectos? Campanópolis, l...","Sun, 13 Feb 2022 17:00:00 GMT","¿Arquitectura sin arquitectos? Campanópolis, l...",https://www.plataformaarquitectura.cl/cl/97602...,Buenos+Aires+Arquitectura,Buenos+Aires,Buenos Aires,0.734651
4,¿Cuáles son los mejores restaurantes de Buenos...,"Thu, 27 Jan 2022 08:00:00 GMT",¿Cuáles son los mejores restaurantes de Buenos...,https://www.lanacion.com.ar/revista-ohlala/cua...,Buenos+Aires+Platos,Buenos+Aires,Buenos Aires,0.725169
...,...,...,...,...,...,...,...,...
72,"Bariloche, un destino ideal para cualquier mom...","Sat, 29 Jan 2022 08:00:00 GMT","Bariloche, un destino ideal para cualquier mom...",https://www.eltiempo.com/vida/viajar/bariloche...,Bariloche+en+que+epoca+ir,Bariloche,Bariloche,0.666394
73,Vacaciones low cost en marzo: opciones y preci...,"Fri, 11 Feb 2022 08:00:00 GMT",Vacaciones low cost en marzo: opciones y preci...,https://www.ambito.com/lifestyle/turismo/vacac...,Bariloche+cuantos+dias+visitar,Bariloche,vacaciones,0.584484
74,Pocos días faltan para la Expo Rural 2022 Bar...,"Mon, 14 Feb 2022 19:20:51 GMT",Pocos días faltan para la Expo Rural 2022 - Ba...,https://www.bariloche2000.com/noticias/leer/po...,Bariloche+Oferta+gastronomica,Bariloche,Bariloche,0.367225
75,Convocan a una asamblea popular por el transpo...,"Fri, 04 Feb 2022 08:00:00 GMT",Convocan a una asamblea popular por el transpo...,https://www.rionegro.com.ar/politica/convocan-...,Bariloche+Transporte,Bariloche,Bariloche,0.652002


In [ ]:
#export to a excel
scrap_df.to_excel("draftdestinos.xlsx", encoding='utf-8')